In [0]:
product_data = [
(2,"samsung","01-01-1995",11000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-01-2006",15000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(3,"oneplus","01-01-2010",23000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
product_df.show()

In [0]:
window = Window.partitionBy("product_id").orderBy('sales_date').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

In [0]:
product_df.withColumn('first_sales',first('sales').over(window))\
            .withColumn('lastest_sales',last('sales').over(window)).explain()

unbounded preceding :- to see records previously <br>
unbounded following - to see records ahead <br>

In [0]:
product_df.withColumn('first_sales',first('sales').over(window))\
            .withColumn('lastest_sales',last('sales').over(window))\
            .withColumn('difference',col('lastest_sales')-col('first_sales'))\
            .select(col('product_id'),col('product_name'),col('difference')).distinct()\
            .show()

Punching time problem

In [0]:
emp_data = [(1,"manish","11-07-2023","10:20"),
        (1,"manish","11-07-2023","11:20"),
        (2,"rajesh","11-07-2023","11:20"),
        (1,"manish","11-07-2023","11:50"),
        (2,"rajesh","11-07-2023","13:20"),
        (1,"manish","11-07-2023","19:20"),
        (2,"rajesh","11-07-2023","17:20"),
        (1,"manish","12-07-2023","10:32"),
        (1,"manish","12-07-2023","12:20"),
        (3,"vikash","12-07-2023","09:12"),
        (1,"manish","12-07-2023","16:23"),
        (3,"vikash","12-07-2023","18:08")]

emp_schema = ["id", "name", "date", "time"]
emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [0]:
emp_df.show()

In [0]:
emp_df = emp_df.withColumn('timestamp',
                  from_unixtime(unix_timestamp(expr("CONCAT(date,' ',time)"),"dd-MM-yyyy HH:mm")))

In [0]:
emp_df.show()

In [0]:
window = Window.partitionBy('id','date').orderBy('date').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)

In [0]:
new_df = emp_df.withColumn('login',first('timestamp').over(window))\
                .withColumn('logout',last('timestamp').over(window))\
                .withColumn('login',to_timestamp('login',"yyyy-MM-dd HH:mm:ss"))\
                .withColumn('logout',to_timestamp('logout','yyyy-MM-dd HH:mm:ss'))\
                .withColumn('duration',(col('logout')-col('login')))\
                .show()

In [0]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]

product_schema=["product_id","product_name","sales_date","sales"]

product_df = spark.createDataFrame(data=product_data,schema=product_schema)

In [0]:
window = Window.partitionBy("product_id").orderBy('sales_date').rowsBetween(-2,0)

In [0]:
window=Window.partitionBy("product_id").orderBy("sales_date").rowsBetween(-2,0)
window1=Window.partitionBy("product_id").orderBy("sales_date")
product_df.withColumn("running_sum",sum("sales").over(window))\
           .withColumn("row_number",row_number().over(window1))\
           .filter(col("row_number")>2)\
           .withColumn("average_sales",round((col("running_sum")/3),2))\
           .show()